# Introduction aux réseaux de neurones en Python

## MLP vs. Deep Learning

Une librairie comme `scikit-learn` n'est pas orientée vers le Deep Learning. Néanmoins, `scikit-learn` dispose de module permettant la construction de réseaux de neurones peu profonds. A l'inverse des librairies comme `tensorflow`, `pytorch` ont été conçues pour le Deep Learning mais permettent aussi d'entraîner des MLP et au-delà (modèles linéaires, SVM, gradient boosting) au prix d'une flexibilité moins grande qu'en Python. Même si les librairies de Deep Learning fournissent des API Python, il se peut que certaines fonctionnalités et de petites modifications ne soient pas simples car le code source est en C++.

## Première version des MLP pour la classification

### Reproduisons le premier exemple du cours



In [ ]:
from sklearn import metrics, datasets

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
iris = datasets.load_iris()

Regardons la description des données

In [ ]:
print ("Iris data set Description :: ", iris['DESCR'])

Il y a 4 variables d'entrée

In [ ]:
X=iris.data

On stocke le numéro de la classe dans y, il varie entre 0 et 2

In [ ]:
y=iris.target

Traçons les données

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')
plt.title('Sepal Width & Length')
plt.show()

Séparons les 150 données en 75 données d'apprentissage et 75 de test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

Construisons un MLP pour la classification des données avec 2 neurones cachés et une pénalisation sur les poids de 0.01

In [ ]:
mlp_iris=MLPClassifier(solver='lbfgs', alpha=0.01,
...                     hidden_layer_sizes=(2,), random_state=1)

In [ ]:
mlp_iris.fit(X_train,y_train)

Faisons maintenant la prédiction sur la base de test

In [ ]:
y_pred=mlp_iris.predict(X_test)

Traçons la matrice de confusion

In [ ]:
cm = metrics.confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n%s" % cm)

**Exercice** : Tester sans pénalisation et avec seulement 25 exemples dans la base d'apprentissage

## Première version des MLP pour la régression

### Reproduisons le deuxième exemple du cours

In [ ]:
from math import *

In [ ]:
import numpy as np

In [ ]:
from sklearn.neural_network import MLPRegressor

Définissons la fonction f1

In [ ]:
def f1(x):
    x1=x[:,0]
    x2=x[:,1]
    x3=x[:,2]
    return np.sin(x1)/x1+0.1*x3**4+2*np.sin(x2)**2

Définissons les données d'entrée qui suivent une loi uniforme entre - Pi et Pi

In [ ]:
x=np.random.uniform(-pi,pi,450)

In [ ]:
x=np.reshape(x,(150,3))

In [ ]:
y=f1(x)

In [ ]:
from sklearn.model_selection import train_test_split

Séparons les 150 données avec 1/4 de données test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

Construction du réseau

In [ ]:
nnet=MLPRegressor(hidden_layer_sizes=(11,),max_iter=1000,activation='tanh',alpha=0.01,solver='lbfgs')

Apprentissage

In [ ]:
nnet.fit(X_train,y_train)

Prédiction

In [ ]:
y_pred=nnet.predict(X_test)

Visualisation

In [ ]:
%pylab inline

In [ ]:
plt.plot(y_pred, y_test,'o')

Calcul du coefficient de détermination R2

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,y_pred)

Validation croisée

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
kf = KFold(n_splits=5, shuffle=True)

In [ ]:
 print(cross_val_score(nnet, x, y, cv=kf))  

**Exercice** : Tester sans pénalisation et avec différents nombres de neurones cachés

### Essai sur la base de chiffres MNIST

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

On importe les données

In [ ]:
mnist_train_data = np.loadtxt("mnist_train.csv", 
                        delimiter=",")
mnist_test_data = np.loadtxt("mnist_test.csv", 
                       delimiter=",") 

Format des données

In [ ]:
print("Image Data Shape" , mnist_train_data.shape)

In [ ]:
print("Label Data Shape", mnist_test_data.shape)

Les données sont des images 28 par 28 - première colonne = label

On visualise une des données

In [ ]:
%pylab inline

In [ ]:
plt.figure()
image=mnist_train_data[150,1:785]
plt.imshow(np.reshape(image, (28,28)), cmap=plt.cm.gray)


In [ ]:
print(mnist_train_data[:,0])

On construit un MLP

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
classif = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(256,), random_state=1, alpha=0.005)

In [ ]:
classif.fit(mnist_train_data[:,1:765], mnist_train_data[:,0])   


In [ ]:
y_pred = classif.predict(mnist_test_data[:,1:765])

In [ ]:
print('Test accuracy:', classif.score(mnist_test_data[:,1:765], mnist_test_data[:,0]))

In [ ]:
from sklearn import metrics

In [ ]:
cm = metrics.confusion_matrix(mnist_test_data[:,0], y_pred)
print("Confusion matrix:\n%s" % cm)

Visualisation avec t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne=TSNE(n_components=2, random_state=1)

In [ ]:
X_2d=tsne.fit_transform(mnist_test_data[0:1000,1:765])

In [ ]:
%pylab inline

In [ ]:
plt.scatter(X_2d[:,0], X_2d[:,1], c=y_pred[0:1000], cmap=plt.cm.get_cmap("jet", 10))
plt.colorbar(ticks=range(10))

In [ ]:
plt.scatter(X_2d[:,0], X_2d[:,1], c=mnist_test_data[0:1000,0], cmap=plt.cm.get_cmap("jet", 10))
plt.colorbar(ticks=range(10))